## Data Exploration

In [365]:
import pandas as pd
import seaborn as sns
import numpy as np
from pandasql import sqldf

In [366]:
#load data
raw_data=pd.read_csv('https://github.com/aaas24/python_projects/raw/master/PUBLISHED/ted_talks/1_raw_data/final_raw_data.csv')
df=raw_data.copy()

### Exploring data

In [367]:
df.dtypes

Unnamed: 0         int64
title             object
author_x          object
date              object
views              int64
likes              int64
link              object
url               object
author_y          object
title_1           object
title_2           object
description_1     object
description2     float64
duration_seg       int64
release_date      object
keywords          object
description_2     object
dtype: object

In [368]:
# missing data
((df
 .isna()
 .mean()
 *100)
 .pipe(lambda ser:ser[ser>0])
)

author_x          0.018382
description2    100.000000
dtype: float64

In [369]:
df.isna().sum()

Unnamed: 0          0
title               0
author_x            1
date                0
views               0
likes               0
link                0
url                 0
author_y            0
title_1             0
title_2             0
description_1       0
description2     5440
duration_seg        0
release_date        0
keywords            0
description_2       0
dtype: int64

### final cleaning columns

In [370]:
#drop columns with duplicated data or not useful
drop_column=['description2', 'Unnamed: 0', 'link', 'author_y', 'title','url', 'title_1']
df2=df.drop(drop_column,axis=1)
df2.head(30)
df_result=df2.copy()

In [371]:
#column names
df_result.columns.values

array(['author_x', 'date', 'views', 'likes', 'title_2', 'description_1',
       'duration_seg', 'release_date', 'keywords', 'description_2'],
      dtype=object)

In [372]:
column_names=['author', 'date_recorded', 'views', 'likes', 'title',
       'description_1', 'duration_seg', 'date_released', 'keywords','description_2']
df_result.columns=column_names
df_result.columns.values

array(['author', 'date_recorded', 'views', 'likes', 'title',
       'description_1', 'duration_seg', 'date_released', 'keywords',
       'description_2'], dtype=object)

In [373]:
colum_names_reordered=['author', 'title','description_1','description_2','date_recorded', 'date_released','views', 'likes', 
        'duration_seg',  'keywords']
df_end=df_result[colum_names_reordered]
df_end.head(5)

,author,title,description_1,description_2,date_recorded,date_released,views,likes,duration_seg,keywords
0,Ozawa Bineshi Albert,Climate action needs new frontline leadership,"""We can't rely on those who created climate ch...","""We can't rely on those who created climate ch...",December 2021,2022-02-18 09:41:43,404000,12000,834,"['TED', ' talks', ' climate change', ' global ..."
1,Sydney Iaukea,The dark history of the overthrow of Hawaii,"""On January 16th, 1895, two men arrived at Lil...","""On January 16th, 1895, two men arrived at Lil...",February 2022,2022-02-17 10:13:22,214000,6400,0,"['TED', ' talks', ' education', ' women', ' TE..."
2,Martin Reeves,Why play is essential for business,"""To thrive in today's competitive economy, you...","""To thrive in today's competitive economy, you...",September 2021,2022-02-17 09:51:37,412000,12000,665,"['TED', ' talks', ' business', ' leadership', ..."
3,James K. Thornton,Why is China appointing judges to combat clima...,"""Why is China appointing thousands of judges t...","""Why is China appointing thousands of judges t...",October 2021,2022-02-16 09:45:34,427000,12000,695,"['TED', ' talks', ' climate change', ' polluti..."
4,Mahendra Singhi,Cement's carbon problem -- and 2 ways to fix it,"""Cement is vital to modernizing all kinds of i...","""Cement is vital to modernizing all kinds of i...",October 2021,2022-02-16 09:38:48,2400,72,671,"['TED', ' talks', ' climate change', ' Countdo..."


In [374]:
df=df_end.copy()
type(df['date_recorded'][0])

str

In [376]:
#modifying the date_recorded variable
df_ini=df_end.copy()
df=pd.DataFrame()
df['date_recorded']=df_ini['date_recorded']
df['date_recorded']= pd.to_datetime(df['date_recorded'], format='%B %Y')
# # Add Column with months
list_months=[]
list_years=[]
for i in range(df.shape[0]):
    list_months.append(df['date_recorded'][i].month)
    list_years.append(df['date_recorded'][i].year)
df['date_recorded_year']=list_years
df['date_recorded_month']=list_months
print(df.head(10))



  date_recorded  date_recorded_year  date_recorded_month
0    2021-12-01                2021                   12
1    2022-02-01                2022                    2
2    2021-09-01                2021                    9
3    2021-10-01                2021                   10
4    2021-10-01                2021                   10
5    2021-10-01                2021                   10
6    2022-02-01                2022                    2
7    2021-08-01                2021                    8
8    2022-02-01                2022                    2
9    2019-03-01                2019                    3


In [377]:
#modifying the date_released variable
df_ini=df_end.copy()
column='date_released'
df=pd.DataFrame()
df[column]=df_ini[column]
df[column]= pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S')

# # Add Column with months
list_months=[]
list_years=[]
list_hours=[]
list_minutes=[]
for i in range(df.shape[0]):
    list_months.append(df[column][i].month)
    list_years.append(df[column][i].year)
    list_hours.append(df[column][i].hour)
    list_minutes.append(df[column][i].minute)
df[column+'_year']=list_years
df[column+'_month']=list_months
df[column+'_hour']=list_hours
df[column+'_minute']=list_minutes

print(df.head(10))



        date_released  date_released_year  date_released_month  \
0 2022-02-18 09:41:43                2022                    2   
1 2022-02-17 10:13:22                2022                    2   
2 2022-02-17 09:51:37                2022                    2   
3 2022-02-16 09:45:34                2022                    2   
4 2022-02-16 09:38:48                2022                    2   
5 2022-02-15 09:29:19                2022                    2   
6 2022-02-14 10:01:51                2022                    2   
7 2022-02-14 09:57:02                2022                    2   
8 2022-02-14 09:53:24                2022                    2   
9 2022-02-07 10:05:47                2022                    2   

   date_released_hour  date_released_minute  
0                   9                    41  
1                  10                    13  
2                   9                    51  
3                   9                    45  
4                   9                    38

### Keywords: 
#### What type of content is published

In [478]:
df=df_end.loc[0:100,"keywords"]
df=str(','.join(df)).replace("[","").replace("]","")
df2=pd.DataFrame(df)
df2.str.get_dummies(',')
print(df)

ValueError: DataFrame constructor not properly called!

In [446]:
def separate_list(df):
    return ','.join(df)
lines=[]
df=df_end.loc[0:100,"keywords"]
for i in range(0, df.shape[0]):
    line=(
        str(df_x[i])
        .replace("[","")
        .replace("]","")
    )
#     lines.append(line)
#     df2["keywords2"]=lines
df2 = pd.DataFrame([line], columns=['keyword2'])
df2=df2["keyword2"].str.get_dummies(',')
print(df2)



# df_result=
# (df['keywords']
#  .apply(separate_list(df['keywords']))
# )
# print(df_result)

    ' Middle East'   ' activism'   ' community'   ' education'   ' equality'  \
0                1             1              1              1             1   

    ' kids'   ' social change'   ' talks'   ' teaching'   ' women'  'TED'  
0         1                  1          1             1          1      1  


In [379]:
from collections import Counter
from itertools import chain

lists = []

for i in range(5430, df.shape[0]):
    line=(df_x[i]
          .replace("[","")
          .replace("]","")
          .replace("'","")
          .split(",")
         )
    lists.append(line)
no_of_lists_per_name = Counter(chain.from_iterable(map(set, lists)))

for name, no_of_lists in no_of_lists_per_name.most_common():
    if no_of_lists == 1:
        break # since it is ordered by count, once we get this low we are done
    print(f"'{name}' is in {no_of_lists} lists")


'TED' is in 10 lists
'  talks' is in 10 lists
'  atheism' is in 5 lists
'  religion' is in 5 lists
'  culture' is in 5 lists
'  humor' is in 3 lists
'  Christianity' is in 3 lists
'  storytelling' is in 3 lists
'  performance' is in 3 lists
'  comedy' is in 3 lists
'  environment' is in 3 lists
'  sustainability' is in 3 lists
'  technology' is in 2 lists
'  global issues' is in 2 lists
'  science' is in 2 lists
'  evolution' is in 2 lists
'  cognitive science' is in 2 lists
'  philosophy' is in 2 lists
'  consciousness' is in 2 lists
'  brain' is in 2 lists
'  business' is in 2 lists
'  politics' is in 2 lists
'  equality' is in 2 lists
'  pollution' is in 2 lists
'  cities' is in 2 lists
'  activism' is in 2 lists


In [380]:
from collections import Counter
from itertools import chain

lists = []
list_fix=[]
for i in range(5430, df.shape[0]):
    line=(df_x[i]
          .replace("[","")
          .replace("]","")
          .replace("'","")
          .replace('\t', '')
          .split(",")
         )
    #recompose the line without spaces
    for word in line:
        list_fix.append(word.replace(' ', '',2))
    lists.append(list_fix)
no_of_lists_per_name = Counter(chain.from_iterable(map(set, lists)))

for name, no_of_lists in no_of_lists_per_name.most_common():
#     if no_of_lists == 1:
#         break # since it is ordered by count, once we get this low we are done
    print(name ,no_of_lists)

education 10
creativity 10
evolution 10
parenting 10
TED Prize 10
pollution 10
cognitive science 10
atheism 10
transportation 10
renewable energy 10
religion 10
science 10
humor 10
politics 10
TED 10
cities 10
climate change 10
equality 10
ebola 10
dance 10
collaboration 10
performance 10
environment 10
storytelling 10
disease 10
consciousness 10
technology 10
brain 10
Christianity 10
sustainability 10
philosophy 10
culture 10
health 10
talks 10
activism 10
teaching 10
comedy 10
kids 10
global issues 10
business 10


In [381]:
import collections

for i in range(5430, df.shape[0]):
    line=(df_x[i]
          .replace("[","")
          .replace("]","")
          .replace("'","") 
          .split(",")
         )
    for word in line:
            print(word.replace(' ', ''))
          


TED
talks
TEDPrize
collaboration
disease
ebola
globalissues
health
science
technology
TED
talks
atheism
brain
cognitivescience
consciousness
evolution
philosophy
religion
TED
talks
atheism
brain
cognitivescience
consciousness
evolution
philosophy
religion
TED
talks
Christianity
atheism
comedy
culture
performance
religion
storytelling
humor
TED
talks
Christianity
atheism
comedy
culture
performance
religion
storytelling
humor
TED
talks
Christianity
atheism
comedy
culture
performance
religion
storytelling
humor
TED
talks
creativity
culture
dance
education
parenting
teaching
kids
TED
talks
activism
business
cities
environment
politics
pollution
sustainability
equality
TED
talks
activism
business
cities
environment
politics
pollution
sustainability
equality
TED
talks
climatechange
culture
environment
globalissues
science
sustainability
technology
transportation
renewableenergy


In [382]:
# from collections import Counter
# df_x=df['keywords'].head(10)
# cnt = Counter()
# for i in range(0, df_x.shape[0]):
#     words=(df_x[i])

In [383]:
# from collections import Counter
# df_x=df['keywords']
# #cnt = Counter()
# list_words=[]
# for i in range(0, df_x.shape[0]):
#     words=df_x[i][1:-1]
#     list_words.append(words)
# #print(str(list_words).replace('"',''))
#     words=str(list_words).replace('"','')
#     cnt[words] += 1
# cnt

# Analysis

In [ ]:
#initialize here the df to utilize. From here on, we will call this df
df=df_result.copy()

In [ ]:
#what constitude a good video based on likes?
y=1000
df_graph=df.likes.apply(lambda x: round(x/y,0))
df_graph.describe()

In [ ]:
#what constitude a good video based on views?
y=1000
df_graph=df.views.apply(lambda x: round(x/y,0))
df_graph.describe()

### Top 10 Liked Videos

In [480]:
#build data: These are the 75% most liked videos
y_var='likes'
df_grap = (
    (df.groupby(['title','author_x','views', 'keywords'])[y_var].sum().reset_index())
    .sort_values([y_var],ascending=[False])
    ).reset_index()
df_grap=df_grap.drop('index', axis=1)
df_grap = df_grap[df_grap[y_var] > 65000]
df_grap

AttributeError: 'str' object has no attribute 'groupby'

In [ ]:
#This is taking the 75% percentile on liked videos
#Modifiable variables
y_var='likes'

#build data
df_grap = (
    (df.groupby(['title_2','author','views'])[y_var].sum().reset_index())
    .sort_values([y_var],ascending=[False])
    ).reset_index()
df_grap=df_grap.drop('index', axis=1)
df_grap = df_grap[df_grap[y_var] > 65000]
df_grap.head(30)

### Top 10 Authors

In [ ]:
#Modifiable variables
y_var='views'
x_var1='author'
x_var2='likes'

#build data
df_grap = (
    (df.groupby([x_var1, x_var2])[y_var].sum().reset_index())
    .sort_values([y_var],ascending=[False])
    ).reset_index().head(10)
df_grap=df_grap.drop('index', axis=1)
df_grap

In [ ]:
#visualize
x_var='author'
y_var='views'
sns.set_theme(style="white")
ax = sns.barplot(x=y_var, y=x_var, data=df_grap, orient="h")

In [ ]:
#visualize
x_var='author'
y_var='likes'
sns.set_theme(style="white")
ax = sns.barplot(x=y_var, y=x_var, data=df_grap, orient="h", palette="Blues_d")

In [ ]:
q = """
SELECT
`title`
FROM
df
LIMIT 5
"""
print(sqldf(q))

Areas of improvements:
    
    1) More information on the authors. Understanding age, gender and nationality of authors, may answer questions related to diversity of the speakers. This data could be parcially scrapped from Wikipedia as there is a dedicated website that tracks this information. 
    https://en.wikipedia.org/wiki/List_of_TED_speakers